In [1]:
from pathlib import Path
import rich
import torch
import transformers

DIR_PATH = Path("/home/mila/g/gagnonju/IteratedDecoding/jobs/tmp/")

pegasus_names = [
    "google/pegasus-pubmed", 
    "google/pegasus-arxiv",
]

bigbird_names = [
    "google/bigbird-pegasus-large-pubmed",
    "google/bigbird-pegasus-large-arxiv",
]

active_model_name = "google/bigbird-pegasus-large-pubmed"

# By default encoder-attention is `block_sparse` with num_random_blocks=3, block_size=64
tokenizer = transformers.AutoTokenizer.from_pretrained(active_model_name)

model = transformers.BigBirdPegasusForConditionalGeneration.from_pretrained(active_model_name)
if torch.cuda.is_available():
    model.to("cuda")

In [2]:
# https://pubmed.ncbi.nlm.nih.gov/30426489/
text = (DIR_PATH / "advances_breast_cancer.txt").read_text().replace("## ", "")
tmp = tokenizer(text, return_tensors='pt')
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
if torch.cuda.is_available():
    inputs.to("cuda")

Token indices sequence length is longer than the specified maximum sequence length for this model (6311 > 4096). Running this sequence through the model will result in indexing errors


1


In [6]:
min_length = 200
max_length = 256
num_beams = 12
num_beam_groups = 12
num_return_sequences = 4

method = "gbs_generate"

if method == "beam_search":
    predictions_tok = model.generate(
        **inputs, 
        repetition_penalty=3., 
        min_length=min_length, 
        num_beams=num_beams,
        max_length=max_length,
        temperature=2.0,
        do_sample=True,
        top_k=num_beams,
        num_return_sequences=num_return_sequences,
        early_stopping=True,
    )

elif method == "gbs_generate":
    predictions_tok = model.generate(
        **inputs, 
        repetition_penalty=3., 
        diversity_penalty=1000.,
        min_length=min_length, 
        num_beams=num_beams,
        max_length=max_length,
        num_beam_groups=num_beam_groups,
        num_return_sequences=num_return_sequences,
        early_stopping=True,
    )
    
elif method == "group_beam_search":
    scorer = transformers.BeamSearchScorer(
        batch_size=1,
        max_length=max_length,
        num_beams=num_beams,
        device="cuda" if torch.cuda.is_available() else "cpu",
        length_penalty=1.0,
        num_beam_hyps_to_keep=num_return_sequences,
        num_beam_groups=num_beam_groups,
    )

    prediction_tok = model.group_beam_search(
        **inputs, 
        repetition_penalty=2., 
        min_length=max_length, 
        diversity_penalty=0.75, 
        beam_scorer=scorer,
    )

else:
    raise ValueError(f"Unknown method: {method}")

for prediction_tok in predictions_tok:
    print(len(prediction_tok))

predictions = tokenizer.batch_decode(
    predictions_tok, truncate=False,
)


/home/mila/g/gagnonju/condaless/lib/python3.8/site-packages/transformers/generation_beam_search.py:196: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(


232
232
232
232


In [8]:
print(f"{len(predictions) = }")
for i, prediction in enumerate(predictions):
    one = prediction.replace("abstract", "[bold]Abstract:[/]\n"
                    ).replace("<n>", ""
                    ).replace("<s>", ""
                    ).replace("<pad>", ""
                    ).replace("</s>", ""
                    ).replace(" - ", "-")

    two = [x.strip().capitalize() for x in one.split(".") if x]
    three = ". ".join(two) + "."

    rich.print(f"[bold]Attempt #{i}[/]: \n{three}")

len(predictions) = 4


Attempt #0: -1878671576393452011
The heterogeneity of breast cancer has been well known for many years. It is now apparent 
that there are distinct genetic subtypes of breast cancer with very similar appearance at 
early stages and different mechanisms of carcinogenesis. This review summarizes the current 
knowledge on molecular features of breast cancer, their relevance to clinicopathological 
aspects and how these may be integrated into therapeutic strategies. The differences in the 
biology of breast cancer between epithelial and mesenchymal cells have been well documented. 
However, the complexity of cell biology makes it difficult to decipher the role of certain 
molecules in breast cancer. Recent studies have highlighted the importance of extracellular 
matrix ( ecm)-responsive tumour-associated proteins. They have also emphasized the need for 
biomarkers that reflect tissue dysfunction. Immunohistochemical staining for a marker or 
combination of markers can identify those patients who are likely to benefit from targeted 
therapies. Analysis of genomic alterations provides information on the mechanism of action by
which drugs affect disease progression. Understanding the role of specific molecules in 
breast cancer will lead to new therapeutics.

Attempt #1: 8308138390663326515
Molecular characterization of breast cancer has advanced significantly over recent decades 
with the discovery of many molecules and their interaction in this disease. These molecules 
are either mutated or have functions associated with tumour biology. The understanding of how
a mutation affects a particular cell type has also increased with the discovery of new 
signalling pathways within that cell type. While some of these mechanisms can be directly 
linked to pathogenesis, others are indirectly determined by the cells themselves. It is now 
well accepted that there is a great deal of heterogeneity within the genetic alterations of 
breast cancer which must be considered when making clinical decisions. This review discusses 
the impact of emerging molecular technologies on the management of breast cancer. It is our 
hope that this article will serve as an initial small step towards achieving more accurate 
diagnosis and improved clinical management of patients with breast cancer through better 
understanding of mutations in genes and their relationship to phenotype. At the end of the 
review, we will discuss the next steps required to further advance the field, including the 
development of prognostic tools.

Attempt #2: -7241039060724225035
Although advanced breast cancer is one of the most common causes for death in women, it has 
been difficult to achieve long-term remission with current standard of care because of 
complex underlying molecular genetic and environmental factors that determine disease 
activity. This review summarizes the progress made in identifying specific genetic 
alterations that contribute to breast cancer development and progression and provide an 
update on the current approaches being used to control tumour growth by targeting known 
pathways. These approaches include gene expression profiling using a panel of over 200 genes,
followed by analysis of mutations at least three times in a cohort of over 500 patients 
before being approved by the us food and drug administration ( fda ) for use in clinical 
practice. There are also presented other studies showing how targeted therapies can be used 
to identify novel molecular targets, which will result in more effective management of 
patients with advanced breast cancer. Recent findings have also demonstrated that certain 
genetic alterations may not always correlate with aggressive phenotypes and that treatment 
response may differ according whether the patient's tumor is deregulated or hyperactivated 
based on the degree of alteration.

Attempt #3: 4804912357575501047
Molecular characterization of breast cancer has advanced significantly over recent decades 
with the discovery of many molecules and their interaction in this disease. These molecules 
are either mutated or have functions associated with tumour biology. The understanding of how
a mutation affects a particular cell type has also increased with the discovery of new 
signalling pathways within that cell type. While some of these mechanisms can be directly 
linked to pathogenesis, others are indirectly determined by the cells themselves. It is now 
well accepted that there is a great deal of heterogeneity within the genetic alterations of 
breast cancer which must be considered when making clinical decisions. This review discusses 
the impact of emerging molecular technologies on the management of breast cancer. It is our 
hope that this article will serve as an initial small step towards achieving more accurate 
diagnosis and improved clinical management of patients with breast cancer through better 
understanding of mutations in genes and their relationship to phenotype. At the end of the 
review, we will discuss the next steps required to further advance the field, including the 
development of prognostic tools. Both genetic and epigenetic techniques for breast cancer 
detection will be reviewed.

In [21]:
help(model.generate)

Help on method generate in module transformers.generation_utils:

generate(input_ids: Union[torch.LongTensor, NoneType] = None, max_length: Union[int, NoneType] = None, min_length: Union[int, NoneType] = None, do_sample: Union[bool, NoneType] = None, early_stopping: Union[bool, NoneType] = None, num_beams: Union[int, NoneType] = None, temperature: Union[float, NoneType] = None, top_k: Union[int, NoneType] = None, top_p: Union[float, NoneType] = None, repetition_penalty: Union[float, NoneType] = None, bad_words_ids: Union[Iterable[int], NoneType] = None, bos_token_id: Union[int, NoneType] = None, pad_token_id: Union[int, NoneType] = None, eos_token_id: Union[int, NoneType] = None, length_penalty: Union[float, NoneType] = None, no_repeat_ngram_size: Union[int, NoneType] = None, encoder_no_repeat_ngram_size: Union[int, NoneType] = None, num_return_sequences: Union[int, NoneType] = None, max_time: Union[float, NoneType] = None, max_new_tokens: Union[int, NoneType] = None, decoder_start_tok